In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
150,AFG,Asia,Afghanistan,2020-06-08,20342,791,357,30,522.550,20.319,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
242,ALB,Europe,Albania,2020-06-08,1246,14,34,0,432.970,4.865,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
398,DZA,Africa,Algeria,2020-06-08,10154,104,707,9,231.557,2.372,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-07,France,154241.0,0.0,29189.0,0.0,0.0,0.0
0,2020-06-07,China,84247.0,0.0,4652.0,0.0,0.0,0.0
0,2020-06-07,Italy,235429.0,0.0,33949.0,0.0,0.0,0.0
0,2020-06-07,Spain,241678.0,0.0,27149.0,0.0,0.0,0.0
0,2020-06-07,United States,1947067.0,0.0,110864.0,0.0,0.0,0.0
0,2020-06-07,World,6964704.0,0.0,403015.0,0.0,0.0,0.0
0,2020-06-07,United Kingdom,286885.0,0.0,40848.0,0.0,0.0,0.0
0,2020-06-07,Germany,184457.0,0.0,8698.0,0.0,0.0,0.0
0,2020-06-07,Iran,172378.0,0.0,8277.0,0.0,0.0,0.0
0,2020-06-07,Turkey,170111.0,0.0,4686.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-08,France,154241.0,153977.0,29189.0,29155.0,264.0,34.0
0,2020-06-08,China,84247.0,84191.0,4652.0,4638.0,56.0,14.0
0,2020-06-08,Italy,235429.0,234998.0,33949.0,33899.0,431.0,50.0
0,2020-06-07,Spain,241678.0,241550.0,27149.0,27136.0,128.0,13.0
0,2020-06-08,United States,1947067.0,1942363.0,110864.0,110514.0,4704.0,350.0
0,2020-06-08,World,6964704.0,6960259.0,403015.0,401970.0,4445.0,1045.0
0,2020-06-08,United Kingdom,286885.0,286194.0,40848.0,40542.0,691.0,306.0
0,2020-06-08,Germany,184457.0,184193.0,8698.0,8674.0,264.0,24.0
0,2020-06-08,Iran,172378.0,171789.0,8277.0,8281.0,589.0,-4.0
0,2020-06-08,Turkey,170111.0,170132.0,4686.0,4692.0,-21.0,-6.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")